<a href="https://colab.research.google.com/github/lalitpagaria/obsei/blob/master/example/Obsei_playstore_classification_logger_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Install latest Obsei

In [1]:
!pip install git+https://github.com/lalitpagaria/obsei.git

  Cloning https://github.com/lalitpagaria/obsei.git to /tmp/pip-req-build-qs53rb_m
  Running command git clone -q https://github.com/lalitpagaria/obsei.git /tmp/pip-req-build-qs53rb_m
  Created wheel for obsei: filename=obsei-0.0.6-cp37-none-any.whl size=50033 sha256=03495c0dc02b38e0495e9fd47c6e4d0d20a694230ef6bf51e172686f4e4e0c12
  Stored in directory: /tmp/pip-ephem-wheel-cache-tulmgikp/wheels/49/1a/6e/2fd83c9a275b7096fc615a0edef2d55b1fc33c3751ba45c1ad
Successfully built obsei


## Configure Play Store Scrapper Source



In [2]:
from obsei.source.playstore_scrapper import PlayStoreScrapperConfig, PlayStoreScrapperSource

# initialize play store source config
source_config = PlayStoreScrapperConfig(
   # Need two parameters package_name and country. 
   # `package_name` can be found at the end of the url of app in play store. 
   # For example - https://play.google.com/store/apps/details?id=com.google.android.gm&hl=en&gl=US
   # `com.google.android.gm` is the package_name for xcode and `us` is country.
   countries=["us"],
   package_name="com.google.android.gm",
   max_count=10, # Number of reviews to fetch
   lookup_period="1h" # Lookup period from current time, format: `<number><d|h|m>` (day|hour|minute)

)

# initialize play store reviews retriever
source = PlayStoreScrapperSource()

## Configure Text Classification Analyzer

In [3]:
from obsei.analyzer.classification_analyzer import ClassificationAnalyzerConfig, ZeroShotClassificationAnalyzer

# initialize classification analyzer config
# It can also detect sentiments if "positive" and "negative" labels are added.
analyzer_config=ClassificationAnalyzerConfig(
   labels=["interface", "crash", "performance"],
)

# initialize classification analyzer
# For supported models refer https://huggingface.co/models?filter=zero-shot-classification
text_analyzer = ZeroShotClassificationAnalyzer(
   model_name_or_path="typeform/mobilebert-uncased-mnli",
   gpu_device=0
)

## Configure Logger Sink

In [4]:
from obsei.sink.logger_sink import LoggerSink, LoggerSinkConfig
import logging
import sys

logger = logging.getLogger("Obsei")
logging.basicConfig(stream=sys.stdout, level=logging.INFO)

# initialize logger sink config
sink_config = LoggerSinkConfig(
   logger=logger,
   level=logging.INFO
)

# initialize logger sink
sink = LoggerSink()

## Execute Workflow/Pipeline

In [5]:
# This will fetch information from configured source
source_response_list = source.lookup(source_config)

# This will execute analyzer
analyzer_response_list = text_analyzer.analyze_input(
    source_response_list=source_response_list,
    analyzer_config=analyzer_config
)

# This will send analyzed output to sink
sink.send_data(analyzer_response_list, sink_config)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


INFO:Obsei:{'processed_text': 'Very safe app.I thank you for alerting of any problems.', 'segmented_data': {'positive': 0.989725649356842, 'interface': 0.7119664549827576, 'performance': 0.25247907638549805, 'crash': 0.012097842991352081, 'negative': 0.0014631063677370548}, 'meta': {'reviewId': 'gp:AOqpTOHapb-uwZEXjZcKD4FpilLyxqAQ7JzlExlrWF1zCOInfSTBeGlgfYRX-Fil6ztpezMjQD5bEhZCpH1rzvs', 'userName': 'Vickie Corneal', 'userImage': 'https://play-lh.googleusercontent.com/-REISJG7Hpl4/AAAAAAAAAAI/AAAAAAAAAAA/AMZuuclGPx3VL-9ZEUxxWDiPHGlA6gWn3A/photo.jpg', 'content': 'Very safe app.I thank you for alerting of any problems.', 'score': 4, 'thumbsUpCount': 0, 'reviewCreatedVersion': '2019.06.09.254811277.release', 'at': datetime.datetime(2021, 3, 26, 22, 22, 15), 'replyContent': None, 'repliedAt': None}, 'source_name': 'PlayStoreScrapper'}
INFO:Obsei:{'processed_text': "Had a new phone (pixel 4a) since Jan 2021 & thought it was time to switch to Gmail app for my emails especially since it's ran 